<a href="https://colab.research.google.com/github/souza-eab/geo-rk/blob/main/FREL_BR_Modify_NatureClimateChange_2025.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# LIBS

In [1]:
#Libs
import pandas as pd
import glob
import os

In [2]:
# Caminho da pasta
path = '/content/drive/MyDrive/Results_V0'


In [3]:
# Listar todos os arquivos CSV na pasta
csv_files = glob.glob(os.path.join(path, "*.csv"))

In [8]:
# Lista para armazenar os DataFrames
dfs = []

# Processar cada arquivo
for file in csv_files:
    # Extrair o nome do arquivo sem a extensão
    filename = os.path.basename(file).replace('.csv', '')

    # Dividir o nome do arquivo em partes
    parts = filename.split('_')

    # Garantir que há pelo menos 5 partes para extrair as variáveis
    if len(parts) >= 5:
        emission_source = parts[2]  # Segunda parte
        type_gas = parts[3]  # Terceira parte
        year = parts[4]  # Quarta parte

        # Carregar o CSV
        df = pd.read_csv(file)

        # Adicionar as novas colunas
        df['emission_source'] = emission_source
        df['type_gas'] = type_gas
        df['year'] = year

        # Adicionar à lista
        dfs.append(df)

# Concatenar todos os DataFrames
df_final = pd.concat(dfs, ignore_index=True)

# Exibir as primeiras linhas
df_final.head()

,system:index,AREA_KM2,Bioma,CD_Bioma,CD_UF,FID_BR_UF_,FID_lm_bio,NM_REGIAO,NM_UF,SIGLA_UF,sum,.geo,emission_source,type_gas,year
0,0000000000000000002a,903208.361,Amazônia,1,51,22,0,Centro-oeste,Mato Grosso,MT,486942.639563,"{""type"":""MultiPoint"",""coordinates"":[]}",def,n2o,2017
1,00000000000000000029,329651.496,Amazônia,1,21,5,0,Nordeste,Maranhão,MA,82224.965385,"{""type"":""MultiPoint"",""coordinates"":[]}",def,n2o,2017
2,0000000000000000002b,237754.172,Amazônia,1,11,25,0,Norte,Rondônia,RO,549228.647120,"{""type"":""MultiPoint"",""coordinates"":[]}",def,n2o,2017
3,0000000000000000002d,164173.429,Amazônia,1,12,0,0,Norte,Acre,AC,132647.773896,"{""type"":""MultiPoint"",""coordinates"":[]}",def,n2o,2017
4,0000000000000000002e,1559255.881,Amazônia,1,13,1,0,Norte,Amazonas,AM,561138.889319,"{""type"":""MultiPoint"",""coordinates"":[]}",def,n2o,2017


#Create - tbl - aux 'Correlation factor by biome

In [7]:
# Criar uma tabela nova por biomas do Brasil e o fc, fc_positivo, fc_negativo


df_aux = df_fc.list[(Amazônia, Amazon, 1.013, 1.019, 1.010, 0.006),
                    (Caatinga, Caatinga, 1.019, 1.024, 1.014, 0.005),
                    (Cerrado, Cerrado, 0.995, 1.001, 0.989, 0.006),
                    (Mata Atlântica,Atlantic Forest, 1, 1.005, 0.995, 0.005),
                    (Pampa, Pampa, 0.996, 1.009, 0.983, 0.013),
                    (Pantanal,Pantanal, 1.002, 1.011, 0.993,0.009)]



SyntaxError: invalid syntax. Perhaps you forgot a comma? (<ipython-input-7-fa9129019f34>, line 7)

# Table 1 - Estimated emissions and results (difference between FREL and the analyzed year) by biome for 2022 and 2023. The percentages in parentheses indicate that positive values represent emissions above the FREL 2017-2021 baseline, while negative values represent emissions below the FREL.

In [ ]:
# Converter a coluna 'ano' para numérico
df_final['ano'] = pd.to_numeric(df_final['ano'])

# Filtrar os anos de 2017 a 2021 e apenas 'co2' no type_gas
df_filtered = df_final[df_final['ano'].between(2017, 2021) & (df_final['type_gas'] == 'co2')]

# Agrupar por bioma e calcular a soma
tbl1 = df_filtered.groupby('Bioma', as_index=False)['sum'].sum()  # Use mean() se quiser média

# Renomear colunas
tbl1.rename(columns={'Bioma': 'Biome', 'sum': 'FREL'}, inplace=True)

# Configurar a exibição de floats sem notação científica
pd.options.display.float_format = '{:,.0f}'.format

# Exibir a tabela
tbl1


,Biome,FREL
0,Amazônia,"2,477,153,122"
1,Caatinga,"128,540,752"
2,Cerrado,"519,741,817"
3,Mata Atlântica,"173,298,111"
4,Pampa,"21,045,477"
5,Pantanal,"30,105,875"


In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

# Configurar o estilo
sns.set_style("whitegrid")

# Agrupar por ano e calcular a soma total
df_agg = df_final.groupby('ano', as_index=False)['sum'].sum()
df_agg['sum_milhoes'] = df_agg['sum'] / 1e6  # Converter para milhões

# Média de 2017 a 2023
media_2017_2023 = df_agg[(df_agg['ano'] >= 2017) & (df_agg['ano'] <= 2021)]['sum_milhoes'].mean()

# Criar a figura
plt.figure(figsize=(10, 6))
ax = sns.barplot(data=df_agg, x='ano', y='sum_milhoes', hue='ano', palette='Reds', legend=False)

# Adicionar rótulos dentro das barras
for p in ax.patches:
    ax.annotate(f"{p.get_height():,.0f}",
                (p.get_x() + p.get_width() / 2, p.get_height() - 30),
                ha='center', va='top', fontsize=10, color='white', fontweight='bold')

# Adicionar linha tracejada para a média
plt.axhline(media_2017_2023, linestyle='--', color='black', linewidth=1.5)
plt.text(df_agg['ano'].min(), media_2017_2023 + 20,
         f'{media_2017_2023:,.0f}', fontsize=11, color='black', fontweight='bold')

# Ajustes nos eixos e rótulos
plt.xlabel('')
plt.ylabel('tons CO₂ eq (milhões)')
plt.ylim(0, df_agg['sum_milhoes'].max() * 1.1)
plt.xticks(rotation=45)

# Adicionar legenda
plt.legend(['Official National FREL'], loc='upper left')

# Exibir o gráfico
plt.show()


In [6]:
import seaborn as sns
import matplotlib.pyplot as plt

# Configurar o estilo
sns.set_style("whitegrid")

# Agrupar por year e calcular a soma total
df_agg = df_final.groupby('year', as_index=False)['sum'].sum()
df_agg['sum_milhoes'] = df_agg['sum'] / 1e6  # Converter para milhões

# Convert 'year' column to numeric
df_agg['year'] = pd.to_numeric(df_agg['year'])

# Média de 2017 a 2023
media_2017_2023 = df_agg[(df_agg['year'] >= 2017) & (df_agg['year'] <= 2021)]['sum_milhoes'].mean()

# Criar a figura
plt.figure(figsize=(10, 6))
ax = sns.barplot(data=df_agg, x='year', y='sum_milhoes', hue='year', palette='Reds', legend=False)

# Adicionar rótulos dentro das barras
for p in ax.patches:
    ax.annotate(f"{p.get_height():,.0f}",
                (p.get_x() + p.get_width() / 2, p.get_height() - 30),
                ha='center', va='top', fontsize=10, color='white', fontweight='bold')

# Adicionar linha tracejada para a média
plt.axhline(media_2017_2023, linestyle='--', color='black', linewidth=1.5)
plt.text(df_agg['year'].min(), media_2017_2023 + 20,
         f'{media_2017_2023:,.0f}', fontsize=11, color='black', fontweight='bold')

# Ajustes nos eixos e rótulos
plt.xlabel('')
plt.ylabel('tons CO₂ eq (milhões)')
plt.ylim(0, df_agg['sum_milhoes'].max() * 1.1)
plt.xticks(rotation=45)

# Adicionar legenda
plt.legend(['Official National FREL'], loc='upper left')

# Exibir o gráfico
plt.show()